# Comparing models using regression test tooling
Users of MedCAT might find themselves with multiple different models (i.e the same model at various stage of training).
But it might be unclear which one performs best for their (or someone else's) specific usecase(s).
This is where the regression test suite built into MedCAT (fully supported starting v1.7.0) can be helpful

The regresion suite tooling has been built with a few objectives in mind
- Ability to generate metrics on a pre-defined _gold standard_ regression suite(s)
- Allow users to compare the metrics of different models
- Provide tooling to make generate regression suites

The general workflow of the built in regression tooling as follows
1. Generate a _gold standard_ regression suite (i.e from MedCATtrainer export)
2. \[Optional\] Create relevant categories of CUIs/TUIs/names of concepts
3. \[Optional\] Split the gold standard into multiple regression suites
4. Run regression tests on one (or many) model(s)

In [3]:
# Install medcat
! pip install medcat==1.7.0 # regression suite fully available starting version 1.7.0
try:
    from medcat.cat import CAT
except:
    print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
    exit()


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip


/Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCATtutorials/tvenv/lib/python3.9/site-packages/medcat/cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Generating _gold stantard_ from MedCATtrainer export
After using MedCATtrainer to annotate a number of documents, you can export said annotations.
You can do so by going to the admin page (in default configuration: `http://localhost:8001/admin/`) and under _API_ click on _Project annotate entitiess_.
You then want to select the _Download_ action and pick the project(s) you wish to import and click _Go_ to download the export data.

For the purposes of this tutorial, we will use some automatically annotated texts.

In [14]:
! DATA_DIR="./data_regr"
DATA_DIR = "./data_regr"
! MCT_EXPORT_URL="https://gist.githubusercontent.com/mart-r/b22f737856039f0ceec6ffa7bc49f348/raw/d66019379228a77bb35d6bd30a45e0c618ef0c34/Keggle_MCT_export.json"
MCT_EXPORT_URL = "https://gist.githubusercontent.com/mart-r/b22f737856039f0ceec6ffa7bc49f348/raw/d66019379228a77bb35d6bd30a45e0c618ef0c34/Keggle_MCT_export.json"
! MCT_EXPORT_FILE=$DATA_DIR"/Keggle_MCT_export.json"
MCT_EXPORT_FILE = DATA_DIR + "/Keggle_MCT_export.json"
# get the MCT export json
! wget -N $MCT_EXPORT_URL -P $DATA_DIR


--2023-03-06 16:26:51--  https://gist.githubusercontent.com/mart-r/b22f737856039f0ceec6ffa7bc49f348/raw/d66019379228a77bb35d6bd30a45e0c618ef0c34/Keggle_MCT_export.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134018 (131K) [text/plain]
Saving to: ‘./data_regr/Keggle_MCT_export.json’

Keggle_MCT_export.j 100%[===================>] 130.88K  --.-KB/s    in 0.02s   

Last-modified header missing -- time-stamps turned off.
2023-03-06 16:26:51 (6.44 MB/s) - ‘./data_regr/Keggle_MCT_export.json’ saved [134018/134018]



The next step would be to convert said MCT export to the regression suite format.
There is a built in runnable to do that.

PS: You may need to pass the `--overwrite` option to the command below if you're running this multiple times and wish the same files to be overwritten

In [15]:
! OUTPUT_YAML=$DATA_DIR"/Keggle_regr_suite_output.yml"
OUTPUT_YAML = DATA_DIR + "/Keggle_regr_suite_output.yml"
# run converter
! python -m medcat.utils.regression.mct_converter $MCT_EXPORT_FILE $OUTPUT_YAML --words 20 20 # words before, words after

Starting to convert export JSON to YAML from file data_regr/Keggle_MCT_export.json
/Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCATtutorials/tvenv/lib/python3.9/site-packages/medcat/utils/regression/mct_converter.py:44: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("Creating regression suite with no model-card / metadata")
Creating regression suite with no model-card / metadata
/Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCATtutorials/tvenv/lib/python3.9/site-packages/medcat/utils/regression/mct_converter.py:45: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("Please consider passing --modelcard <model card json> or")
Please consider passing --modelcard <model card json> or
/Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCATtutorials/tvenv/lib/python3.9/site-packages/medcat/utils/regression/mct_converter.py:46: DeprecationWarning: The 'warn' method is deprecated, use 'warning' in

# \[Optional\] Creating category files
We now need to create a category file.
Each category will correspond to a separate regression test set.
So we might want to, for example, separate different domains.

In this case, we will use a somewhat arbitrary categories file.
All the CUIs in the example were simply split into a number of different categories based on the CUIs and TUIs available.

In [16]:
! CATEGORIES_YML=$DATA_DIR"/categories.yml"
CATEGORIES_YML =  DATA_DIR + "/categories.yml"
! OUTPUT_SPLIT=$DATA_DIR"/split-"
OUTPUT_SPLIT = DATA_DIR + "/split-"
! RAW_CATEGORIES_URL="https://gist.githubusercontent.com/mart-r/cbcb231a17ce55a0b5cfbec419b168f5/raw/1f9a058abbe15ffbe3a8c1277d6166ab5f1983b5/categories.yml"
RAW_CATEGORIES_URL = "https://gist.githubusercontent.com/mart-r/cbcb231a17ce55a0b5cfbec419b168f5/raw/1f9a058abbe15ffbe3a8c1277d6166ab5f1983b5/categories.yml"

# Downlaod categories file
! wget -N $RAW_CATEGORIES_URL -P $DATA_DIR

# Separate into categories
! python -m medcat.utils.regression.category_separator $CATEGORIES_YML $OUTPUT_YAML $OUTPUT_SPLIT- --strategy FIRST

--2023-03-06 16:28:33--  https://gist.githubusercontent.com/mart-r/cbcb231a17ce55a0b5cfbec419b168f5/raw/1f9a058abbe15ffbe3a8c1277d6166ab5f1983b5/categories.yml
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6497 (6.3K) [text/plain]
Saving to: ‘./data_regr/categories.yml’

categories.yml      100%[===================>]   6.34K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2023-03-06 16:28:34 (28.8 MB/s) - ‘./data_regr/categories.yml’ saved [6497/6497]



# Run regression tests on one (or many) model(s)

We are now ready to the regression tests on a model.
All we need is a SNOMED based MedCAT model.
You can download the 1.4 model from [here](https://uts.nlm.nih.gov/uts/login?service=https:%2F%2Fmedcat.rosalind.kcl.ac.uk%2Fauth-callback).

PS:
**Make sure to insert the model pack path below in the code if you have your own model**

In [34]:
# we will use the MedMentions model for an example here
! wget -N https://medcat.rosalind.kcl.ac.uk/media/medmen_wstatus_2021_oct.zip -P $DATA_DIR

# ENTER MODEL PATH for custom model
! MODEL_PACK_PATH=$DATA_DIR"/medmen_wstatus_2021_oct.zip"
MODEL_PACK_PATH = DATA_DIR + "/medmen_wstatus_2021_oct.zip"

Now, we need to run the regression suite.
You will need to run one of the two commands below depending on whether or not you split the regression suite into multiple parts.
Since the splitting is enabled by default, the other command is commented out for now.

In [36]:
! echo $OUTPUT_SPLIT"*"
# ! ls $OUTPUT_SPLIT"*"
print(OUTPUT_SPLIT)
# Run regression tests on one (or many) model(s)
# ! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH $OUTPUT_SPLIT"*"
# ! python -m medcat.utiil.
# ! SPLIT_FILES=$OUTPUT_SPLIT"*"
# SPLIT_FILES = OUTPUT_SPLIT + "*"
# ! echo python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH $SPLIT_FILES
# ! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH $SPLIT_FILES
! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH ./data_regr/split--_attributes.yml
! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH ./data_regr/split--_diagnosis.yml
! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH ./data_regr/split--_disorders.yml
! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH ./data_regr/split--_findings-procedures.yml
! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH ./data_regr/split--_medicines.yml
! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH ./data_regr/split--_objects-and-other.yml
! python -m medcat.utils.regression.regression_checker $MODEL_PACK_PATH ./data_regr/split--_qualifiers.yml
# or for the ONE regression suite that was downloaded
# ! python -m medcat.utils>regression.regression_checker.py $MODEL_PACK_PATH $OUTPUT_YAML

./data_regr/split-*
./data_regr/split-
WITH ARGUMENTS:
MPD ../../../working_with_cogstack/data/medcat_models/modelpack/20221004_trained_model_supervised_latest_acd0dfc2f0df45de.zip
TSF data_regr/split--_attributes.yml
SYS ARGS: ['../../../working_with_cogstack/data/medcat_models/modelpack/20221004_trained_model_supervised_latest_acd0dfc2f0df45de.zip', './data_regr/split--_attributes.yml']
Loading RegressionChecker from yaml: data_regr/split--_attributes.yml
Loading model pack from file: ../../../working_with_cogstack/data/medcat_models/modelpack/20221004_trained_model_supervised_latest_acd0dfc2f0df45de.zip
/Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCATtutorials/tvenv/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current

Now, if you have multiple models, you can compare the results of different models.
There are ways to somewhat automate it, but they are not within the scope of this tutorial.
Check out <TODO - mlflow stuff>, instead.